In [1]:
import datetime
import os
import random
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

import torchvision.models as models_vision
import torchaudio.models as models_audio

import core.config as conf
from torchvision.transforms import transforms

from fn_networks import BackboneAud, SubnetAud, SubnetVid, show_feature_map

c:\Users\umari\anaconda3\envs\DL38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


network path c:\Users\umari\projects\AV-spatial-coherence\results\checkpoints\MC_full_vgg_freeze_BCE


In [12]:
class AVOL_Net(nn.Module):

    def __init__(self, 
    heatmap=conf.dnn_arch['heatmap'], 
    inference = conf.training_param['inference']
    ):
        
        self.inference = inference

        super().__init__()
        self.heatmap = heatmap
        
        self.VideoNet = models_vision.vgg11(pretrained=True).features[0:20]

        for layer in self.VideoNet:
                if isinstance(layer, nn.Conv2d):
                    layer.requires_grad_ = False

        self.AudioNet = BackboneAud()

        self.VideoMerge = SubnetVid(BN=False)
        self.AudioMerge = SubnetAud(BN=False)

        self.conv_final = nn.Conv2d(1,1,kernel_size=1)

    def forward(self, x_in, y_in, cam):

        x1 = self.VideoNet(x_in)                                                # 512 x 7 x 7 
        y1 = self.AudioNet(y_in)                                                # 512 x H x W 

        x2 = self.VideoMerge(x1)                                                # 128 x 7 x 7 
        y2 = self.AudioMerge(y1, cam_ID=cam).unsqueeze(-1).unsqueeze(-1)        # 128 x 1 x 1 

        x2 = x2*y2
        
        x2 = x2.mean(dim=1, keepdim=True)                                       # 1 x 14 x 14 

        x2 = self.conv_final(x2).squeeze(1)

        x_map = torch.sigmoid(x2)                                               # 1 x 14 x 14 

        if self.inference:
            features_ls = [
                (x_in, 'input image'),
                (x1, 'after img backbone'),
                (x2, 'after attention'),
                (x_map, 'heatmap')
            ]           
            show_feature_map(features_ls)

        x_class = torch.amax(x_map, (1,2))

        return x_class, x_map

In [13]:
net = AVOL_Net()

Custom for Audio


In [14]:
img = torch.randn((1, 3, 224, 224))
aud = torch.randn((1, 16, 960, 64))
cam = torch.randn((1, 11))

In [15]:
out = net(img, aud, cam)

In [16]:
for o in out:
    print(o.shape)

torch.Size([1])
torch.Size([1, 14, 14])


In [4]:
word = 'abcde-cam08'

word = word[:-word.find('cam')]

word

'abcde'